In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_excel("CITY.xlsx",sheet_name="DATA")

In [3]:
data = data.replace("-",np.nan)

In [4]:
data

,L2,L4_gfcid,Product,Transaction_ID,Transaction_Amount,Fixed_Fee,Adval_Fee,Min,Max,Current_Fee_Type,Proposed_Pricing,Proposed_Min,Proposed_Max,Percentage_Based,Fixed_Fee_Based
0,1,15,NEFT,497,87226,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,30.0
1,1,15,NEFT,875,86409,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN
2,1,15,NEFT,554,94454,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN
3,1,15,NEFT,654,56434,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN
4,1,15,,736,53934,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN
5,1,15,RTGS,675,91527,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN
6,1,15,RTGS,875,73695,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN
7,1,15,RTGS,133,81983,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN
8,1,23,RTGS,336,73966,30.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN
9,1,23,RTGS,538,93845,30.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN


In [5]:
def  selectUniques(data ,dropna =  False ):
    uni_dic= {}
    for  col in data.columns:
        uni   = data[col].unique()
        if dropna == True:
            print(type(uni),col)
            uni =  uni[np.isfinite(uni)]
        uni_dic[col]  = uni
        #print(col)
    return uni_dic;#save this value to object variable while creating class

def isUnique(data,mapDf =  False):#non dynamic function
    uniDf = data[["Min","Max","Adval_Fee"]].drop_duplicates().dropna()
    if mapDf == True :
        return uniDf
    return  uniDf.shape[0] == uniDf.Adval_Fee.unique().shape[0]

def recheck(data,Transaction_Amount):
    t = isUnique(data,mapDf =  True)#[isUnique(data,mapDf =  True)["Adval_Fee"]==0.01]
    temp  = pd.Series(selectUniques(data)["Adval_Fee"]*Transaction_Amount)
    tem2 = pd.Series(selectUniques(data)["Adval_Fee"])
    te = pd.concat([tem2,temp],axis=1)
    te.columns =  ["Adval","Price"]
    
    lookup = pd.merge(te,t,left_on="Adval",right_on="Adval_Fee")
    
    conditions = [
        (lookup['Price'] < lookup['Min']),
        (lookup['Price'] > lookup['Max']),
    ]#
     
    # create a list of the values we want to assign for each condition
    values = [lookup['Min'],lookup['Max']]
    
    # create a new column and use np.select to assign values to it using our lists as arguments
    lookup['final_price'] = np.select(conditions, values,lookup['Price'])
    return lookup

def minMaxMapping(data):
    pass
    


def decisionFunc(Product,Current_Fee_Type,Transaction_Amount,Fixed_Fee,Adval_Fee,Max,Min): 
    '''
        Listing out important columns 
        ["Product","Current_Fee_Type","Transaction_Amount","data.Proposed_Pricing"]
    '''
    if Current_Fee_Type == "Fixed Fee":
        print(Fixed_Fee,[Fixed_Fee  for i in selectUniques(data)["Adval_Fee"][np.isfinite(selectUniques(data)["Adval_Fee"])]])
        return [Fixed_Fee  for i in selectUniques(data)["Adval_Fee"][np.isfinite(selectUniques(data)["Adval_Fee"])]]#Fixed_Fee
    elif Current_Fee_Type == "Adval":
        return list(recheck(data,Transaction_Amount)["final_price"])

        
        
        #temp = isUnique(data,mapDf =  True)
        # amount  =  Transaction_Amount*selectUniques(data)["Adval_Fee"]
        #return [Min if value < Min else Max if value > Max else value for value in amount ]
    
result  =  data.apply(lambda x: decisionFunc(x.Product,x.Current_Fee_Type,x.Transaction_Amount,x.Fixed_Fee,x.Adval_Fee,x.Max,x.Min), axis=1,result_type='expand')
result.columns = ["percent"+str(i)+"_Based" for i in selectUniques(data)["Adval_Fee"][np.isfinite(selectUniques(data)["Adval_Fee"])]]#remove null columns
result = pd.concat([data,result],axis=1)
result
    

    
    
    
    

10.0 [10.0, 10.0]
10.0 [10.0, 10.0]
10.0 [10.0, 10.0]
10.0 [10.0, 10.0]
25.0 [25.0, 25.0]
25.0 [25.0, 25.0]
25.0 [25.0, 25.0]
25.0 [25.0, 25.0]
30.0 [30.0, 30.0]
30.0 [30.0, 30.0]
100.0 [100.0, 100.0]
100.0 [100.0, 100.0]
100.0 [100.0, 100.0]
14.0 [14.0, 14.0]
14.0 [14.0, 14.0]


,L2,L4_gfcid,Product,Transaction_ID,Transaction_Amount,Fixed_Fee,Adval_Fee,Min,Max,Current_Fee_Type,Proposed_Pricing,Proposed_Min,Proposed_Max,Percentage_Based,Fixed_Fee_Based,percent0.01_Based,percent0.02_Based
0,1,15,NEFT,497,87226,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,30.0,10.0,10.0
1,1,15,NEFT,875,86409,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN,10.0,10.0
2,1,15,NEFT,554,94454,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN,10.0,10.0
3,1,15,NEFT,654,56434,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN,10.0,10.0
4,1,15,,736,53934,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
5,1,15,RTGS,675,91527,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
6,1,15,RTGS,875,73695,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
7,1,15,RTGS,133,81983,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
8,1,23,RTGS,336,73966,30.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,30.0,30.0
9,1,23,RTGS,538,93845,30.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,30.0,30.0


In [227]:
selectUniques(data)["Adval_Fee"][np.isfinite(selectUniques(data)["Adval_Fee"])]

array([0.01, 0.02])

conditions = [
    (data['likes_count'] <= 2),
    (data['likes_count'] > 2) & (data['likes_count'] <= 9),
    (data['likes_count'] > 9) & (data['likes_count'] <= 15),
    (data['likes_count'] > 15)
    ]

# create a list of the values we want to assign for each condition
values = ['tier_4', 'tier_3', 'tier_2', 'tier_1']

# create a new column and use np.select to assign values to it using our lists as arguments
data['tier'] = np.select(conditions, values)


,L2,L4_gfcid,Product,Transaction_ID,Transaction_Amount,Fixed_Fee,Adval_Fee,Min,Max,Current_Fee_Type,Proposed_Pricing,Proposed_Min,Proposed_Max,Percentage_Based,Fixed_Fee_Based,percent0.01_Based,percent0.02_Based
0,1,15,NEFT,497,87226,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,30.0,10.0,10.0
1,1,15,NEFT,875,86409,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN,10.0,10.0
2,1,15,NEFT,554,94454,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN,10.0,10.0
3,1,15,NEFT,654,56434,10.0,NaN,NaN,NaN,Fixed Fee,12.00,NaN,NaN,NaN,NaN,10.0,10.0
4,1,15,,736,53934,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
5,1,15,RTGS,675,91527,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
6,1,15,RTGS,875,73695,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
7,1,15,RTGS,133,81983,25.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,25.0,25.0
8,1,23,RTGS,336,73966,30.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,30.0,30.0
9,1,23,RTGS,538,93845,30.0,NaN,NaN,NaN,Fixed Fee,50.00,NaN,NaN,NaN,NaN,30.0,30.0


['percent0.01_Based', 'percent0.02_Based']

In [89]:
def percentage_calculator(df):
    df['new column name'] = df['column name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')

    data["percent_rate"] = data["Transaction_Amount"].apply(lambda tAmt:tAmt*)

SyntaxError: invalid syntax (<ipython-input-89-519d2f02ee09>, line 2)

In [27]:
selectUniques(data)["Adval_Fee"][np.isfinite(selectUniques(data)["Adval_Fee"])]

array([0.01, 0.02])

In [ ]:
for i in  selectUniques(data)["Adval_Fee"][np.isfinite(selectUniques(data)["Adval_Fee"])]:
    data["percent_"+str(i)+"_based"] = data["Transaction_Amount"]*i
    
        